In [1]:
import numpy as np
import pandas as pd
import os
import torch
import transformers

from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
access_token = os.getenv('HUGGINGFACE_TOKEN')

In [3]:
# Load model directly
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B", token=access_token)
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.1-8B", token=access_token)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
print(torch.cuda.get_device_name(torch.cuda.current_device()))

NVIDIA GeForce RTX 3090 Ti


In [14]:
# Move the model to the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define a function to generate text
def generate_text(prompt):
    inputs = tokenizer.encode_plus(
        prompt,
        add_special_tokens=True,
        return_attention_mask=True,
        max_length=10000,
        return_tensors='pt'
    )

    # Move the input tensors to the GPU
    inputs['input_ids'] = inputs['input_ids'].to(device)
    inputs['attention_mask'] = inputs['attention_mask'].to(device)

    outputs = model.generate(
        inputs['input_ids'],
        max_length=10000,
        attention_mask=inputs['attention_mask']
        
    )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

In [8]:
with open('../Datasets/all_prompts_random50.json', 'r') as f:
    text = f.read()

In [15]:
# Test the function
prompt = "Give me a travel plan in Philadelphia for 2 days, given the information of the following business" + text
generated_text = generate_text(prompt)
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Give me a travel plan in Philadelphia for 2 days, given the information of the following business[
    {
        "Description": "Accommodations in Philadelphia",
        "Content": "'name', 'address','stars', 'price'. Four Points by Sheraton Philadelphia Airport,4101A Island Ave,2.5,$$. The Inn at The Union League,1450 Sansom,3.5,$$$. The Rittenhouse Hotel,210 W Rittenhouse Sq,4.0,$$$. Thomas Bond House,129 S 2nd St,4.5,Unknown price. North American Motor Inns,4444 City Ave,1.0,$. Kimpton Hotel Palomar Philadelphia,117 S 17th St,4.0,$$$. Microtel Inn & Suites by Wyndham Philadelphia Airport,8840 Tinicum Boulevard,2.0,$. AKA University City,2929 Walnut St,4.0,Unknown price. Kimpton Hotel Monaco Philadelphia,433 Chestnut St,4.0,$$$. The Warwick Hotel Rittenhouse Square,220 S 17th St,3.0,$$$. Four Points by Sheraton Philadelphia City Center,1201 Race St,3.0,$$. Four Seasons Hotel Philadelphia at Comcast Center,1 N 19th St,4.0,Unknown price. Fairfield Inn by Marriott Philadelphia Airport,8

In [16]:
with open("test_output.txt", "w") as f:
    f.write(generated_text)